In [427]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load the dataset
csv_file = 'final.csv'  # Ensure this file contains the 'Labels' column
cvs2 = 'training_set_z_score_normalized.csv'
data = pd.read_csv(csv_file)
data2 = pd.read_csv(cvs2)

dataset = data.drop('day_id', axis=1)
# dataset = dataset.drop('uv_index', axis=1)
# dataset = dataset.drop('feels_like_celsius', axis=1)
# dataset = dataset.drop('air_quality_us-epa-index', axis=1)
# dataset = dataset.drop('pressure_mb', axis=1)
# dataset = dataset.drop('sunrise', axis=1)
# dataset = dataset.drop('sunset', axis=1)


dataset2 = data2.drop('day_id', axis=1)
dataset2 = dataset2.drop('uv_index', axis=1)
dataset2 = dataset2.drop('city_id', axis=1)
dataset2 = dataset2.drop('feels_like_celsius', axis=1)
dataset2 = dataset2.drop('air_quality_us-epa-index', axis=1)
dataset2 = dataset2.drop('pressure_mb', axis=1)
dataset2 = dataset2.drop('sunrise', axis=1)
dataset2 = dataset2.drop('sunset', axis=1)


In [428]:
dataset.head()

,condition_text,temperature_celsius,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,...,city_id_C103,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112
0,6,-0.325687,-0.976376,-1.491115,-0.345813,3.808121,1.160612,0.802314,-0.442245,0.187573,...,0,0,0,0,0,0,0,0,0,0
1,0,-0.791186,-0.634911,-0.084787,0.802218,-0.246614,0.506188,-1.269057,-0.791270,0.187573,...,0,0,0,0,0,0,0,0,0,0
2,1,-0.170521,-0.921742,1.522446,0.310205,-0.246614,1.160612,0.111857,-0.325903,0.187573,...,0,0,0,0,0,0,0,0,0,0
3,0,-0.480854,-0.976376,-0.948674,0.638214,-0.246614,0.506188,-1.269057,-0.558586,0.187573,...,0,0,0,0,0,0,0,0,0,0
4,1,-0.480854,-0.976376,-0.757815,-0.509818,-0.246614,-0.311842,-0.578600,-0.558586,0.187573,...,0,0,0,0,0,0,0,0,0,0


In [429]:
# Import necessary libraries
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader

# Load your dataset
# Assuming your DataFrame is named 'df'
# df = pd.read_csv('your_dataset.csv')

# Preprocess your data
# Handle missing values
# df = df.dropna()

# Convert categorical data to numerical data

# le = LabelEncoder()
# df['condition_text'] = le.fit_transform(df['condition_text'])

# Split your data into features (X) and target (y)
# x = dataset.drop('condition_text', axis=1).values
# y = dataset['condition_text'].values

x = dataset2.drop('condition_text', axis=1).values
y = dataset2['condition_text'].values


In [430]:
print(x.shape)
print(type(x))
# print(x)

(479, 8)
<class 'numpy.ndarray'>


In [431]:
X = torch.from_numpy(x).type(torch.float32)
Y = torch.from_numpy(y).type(torch.float32)

In [432]:
print(X.shape)
print(Y.shape)

torch.Size([479, 8])
torch.Size([479])


In [433]:
print(X[:2])

tensor([[-0.3257, -0.9764, -1.4911,  3.8081,  1.1606,  0.8023,  0.1876, -0.4661],
        [-0.7912, -0.6349, -0.0848, -0.2466,  0.5062, -1.2691,  0.1876, -0.5943]])


In [434]:
# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [435]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

383 383
96 96


In [436]:
# Create a DataLoader
# train_data = TensorDataset(X_train, y_train)
# train_loader = DataLoader(train_data, batch_size=10)

# Define your neural network architecture
class WeatherNetV1(nn.Module):
    def __init__(self, in_features=X_train.shape[1], out_features=len(dataset['condition_text'].unique())):
        super().__init__()
        self.layer_1 = nn.Linear(in_features, 64)
        self.layer_2 = nn.Linear(64, 32)
        self.layer_3 = nn.Linear(32, out_features)

    def forward(self, x_):
        x_ = torch.relu(self.layer_1(x_))
        x_ = torch.relu(self.layer_2(x_))
        x_ = self.layer_3(x_)
        return x_

class WeatherNetV2(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(input_shape, hidden_units),
            nn.Sigmoid(),
            nn.Linear(hidden_units, hidden_units),
            nn.Sigmoid(),
            nn.Linear(hidden_units, output_shape),
        )

    def forward(self, x):
        return self.layer_stack(x)


In [437]:
# model = WeatherNetV1()
model = WeatherNetV2(X_train.shape[1], 128, len(dataset['condition_text'].unique()))
# Define a loss function and an optimizer
lossFn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)


In [438]:
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    corr = torch.eq(y_true, y_pred).sum().item()
    acc = (corr / len(y_pred)) * 100
    return acc


In [439]:
logits = model(X_train).squeeze()
print(logits)
output = torch.sigmoid(logits)
print(output)
print(output.argmax(dim=1))


tensor([[-0.0014,  0.0558, -0.3578,  ..., -0.4728, -0.0492, -0.4628],
        [-0.0049,  0.0385, -0.3533,  ..., -0.4575, -0.0409, -0.4701],
        [-0.0083,  0.0438, -0.3351,  ..., -0.4640, -0.0506, -0.4755],
        ...,
        [-0.0175,  0.0462, -0.3285,  ..., -0.4548, -0.0320, -0.4844],
        [ 0.0136,  0.0679, -0.3637,  ..., -0.4689, -0.0651, -0.4565],
        [ 0.0096,  0.0657, -0.3455,  ..., -0.4918, -0.0643, -0.4772]],
       grad_fn=<SqueezeBackward0>)
tensor([[0.4997, 0.5140, 0.4115,  ..., 0.3840, 0.4877, 0.3863],
        [0.4988, 0.5096, 0.4126,  ..., 0.3876, 0.4898, 0.3846],
        [0.4979, 0.5110, 0.4170,  ..., 0.3860, 0.4873, 0.3833],
        ...,
        [0.4956, 0.5116, 0.4186,  ..., 0.3882, 0.4920, 0.3812],
        [0.5034, 0.5170, 0.4101,  ..., 0.3849, 0.4837, 0.3878],
        [0.5024, 0.5164, 0.4145,  ..., 0.3795, 0.4839, 0.3829]],
       grad_fn=<SigmoidBackward0>)
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3

In [440]:
# l = lossFn(model(X_train), y_train)
y_train = y_train.type(torch.int64)
l = lossFn(model(X_train), y_train)
print(X_train.shape, X_train.dtype)
print(y_train.shape, y_train.dtype)

torch.Size([383, 8]) torch.float32
torch.Size([383]) torch.int64


**BATCH TRAIN**

In [441]:
# torch.manual_seed(42)
# 
# # Train your model
# for epoch in range(50):
#     for batch in train_loader:
#         X_batch, y_batch = batch
#         optimizer.zero_grad()
#         output = model(X_batch)
#         loss = criterion(output, y_batch)
#         loss.backward()
#         optimizer.step()
# 
# # Evaluate your model
# with torch.no_grad():
#     output = model(X_test)
#     _, predicted = torch.max(output, 1)
#     correct = (predicted == y_test).sum().item()
#     print('Accuracy: %.2f' % (correct / len(y_test) * 100))

**WITHOUT BATCH**

In [442]:
torch.manual_seed(42)

device = 'cpu'

X_train, y_train = X_train.to(device), y_train.to(device).type(torch.int64)
X_test, y_test = X_test.to(device), y_test.to(device).type(torch.int64)

epochs = 150

print(y_train.dtype)

for epoch in range(epochs):
    model.train()
    
    output = model(X_train)
    y_logits = output.squeeze()
    
    #y_logits_ = y_logits.type(torch.long)
    
    y_pred = torch.sigmoid(y_logits).argmax(dim=1)
    

    loss = lossFn(y_logits, y_train)
    acc = accuracy_fn(y_train, y_pred)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.inference_mode():
        test_output = model(X_test)
        test_logits = test_output.squeeze()
        test_pred = torch.sigmoid(test_logits).argmax(dim=1)
        
        test_loss = lossFn(test_logits, y_test)
        test_acc = accuracy_fn(y_test, test_pred)
        
        print(f'Epoch {epoch + 1}/{epochs} | Train Loss: {loss:.4f} | Train Accuracy: {acc:.2f}% | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%')
    # if epoch % 10 == 0:
    #     with torch.no_grad():
    #         output = model(X_test)
    #         predicted = torch.max(output, 1)
    #         correct = (predicted == y_test).sum().item()
    #         print('Accuracy: %.2f' % (correct / len(y_test) * 100))


torch.int64
Epoch 1/150 | Train Loss: 2.1208 | Train Accuracy: 10.70% | Test Loss: 1.9704 | Test Accuracy: 27.08%
Epoch 2/150 | Train Loss: 1.9251 | Train Accuracy: 25.07% | Test Loss: 1.9840 | Test Accuracy: 22.92%
Epoch 3/150 | Train Loss: 1.9045 | Train Accuracy: 26.11% | Test Loss: 2.0249 | Test Accuracy: 22.92%
Epoch 4/150 | Train Loss: 1.9188 | Train Accuracy: 26.11% | Test Loss: 2.0313 | Test Accuracy: 22.92%
Epoch 5/150 | Train Loss: 1.9108 | Train Accuracy: 26.11% | Test Loss: 2.0172 | Test Accuracy: 22.92%
Epoch 6/150 | Train Loss: 1.8925 | Train Accuracy: 26.11% | Test Loss: 2.0014 | Test Accuracy: 25.00%
Epoch 7/150 | Train Loss: 1.8804 | Train Accuracy: 26.89% | Test Loss: 1.9850 | Test Accuracy: 29.17%
Epoch 8/150 | Train Loss: 1.8733 | Train Accuracy: 32.11% | Test Loss: 1.9648 | Test Accuracy: 34.38%
Epoch 9/150 | Train Loss: 1.8638 | Train Accuracy: 39.69% | Test Loss: 1.9409 | Test Accuracy: 38.54%
Epoch 10/150 | Train Loss: 1.8492 | Train Accuracy: 41.51% | Test Loss